### Source:

https://github.com/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_opencellid_demo.ipynb

# OpenCellID Data Exploration with Pandas (accelerated by cudf.pandas) <a target="_blank" href="https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_opencellid_demo.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>

[OpenCellID](https://wiki.opencellid.org/wiki/What_is_OpenCellID) is the world's largest collaborative community project that collects GPS positions of cell towers, used free of charge, for a multitude of commercial and private purposes.

The OpenCellID project was primarily created to serve as a data source for GSM localisation. As of October, 2017, the database contained almost 36 million unique GSM Cell IDs. More than 75,000 contributors have already registered with OpenCellID, contributing millions of new measurements every day in average to the OpenCellID database.

OpenCellID provides 100% free Cell ID data under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/). The OpenCellID database is published under an open content license with the intention of promoting free use and redistribution of the data. All data uploaded by any of the contributors can also be downloaded again free of charge - no exceptions!

# Get Latest RAPIDS

## Load cudf.pandas extension

In [1]:
%load_ext cudf.pandas

In [2]:
import pandas as pd
pd, pd.__version__

(<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>, '2.2.3')

## Import required Packages

In [3]:
# Install required packages
!pip install -q hvplot pydeck panel holoviews

In [4]:
# importing viz libraries
import hvplot.pandas
import pydeck as pdk
import panel as pn
import param
import time
pn.extension("deckgl", loading_indicator=True, template='material')

## Download cell-data (https://www.opencellid.org/)

### Dataset Options
1. [Worldwide Dataset](https://data.rapids.ai/cudf/datasets/cell_towers.tar.xz)
2. [US Dataset](https://data.rapids.ai/cudf/datasets/cell_towers_us.tar.xz): Suitable for the free tier of Google Colab.
### Usage in Google Colab
- For a smaller dataset to test the notebook with the free tier of Google Colab, use: download_and_extract('us').
### Additional Information
- Users can register for an account on OpenCellID to obtain a data access token and download the latest dataset directly.
- The auto-downloader provided in this notebook will not fetch the latest dataset from OpenCellID. For the latest data, manual download with an access token is required.
- If the latest data is not a priority, the included dataset dated May 2024 will suffice for exploring the notebook's functionalities.


In [5]:
# For Google Colab, download the script from a GitHub repository by uncommenting the line below
!wget https://raw.githubusercontent.com/rapidsai-community/showcase/main/getting_started_tutorials/opencellid_downloader.py

--2025-02-03 16:39:10--  https://raw.githubusercontent.com/rapidsai-community/showcase/main/getting_started_tutorials/opencellid_downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
connected. to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 
200 OKequest sent, awaiting response... 
Length: 2341 (2.3K) [text/plain]
Saving to: ‘opencellid_downloader.py.3’

opencellid_download 100%[===================>]   2.29K  --.-KB/s    in 0s      

2025-02-03 16:39:10 (45.5 MB/s) - ‘opencellid_downloader.py.3’ saved [2341/2341]



In [6]:
from opencellid_downloader import download_and_extract
download_and_extract('us') # use download_and_extract('worldwide') to download the entire world dataset

opencellid_data/cell_towers_us.csv already exists. Skipping download and extraction.


## Read cell-data (https://www.opencellid.org/)

### Column Descriptions

- **Radio**:	The generation of broadband cellular network technology (Eg. LTE, GSM)

- **MCC**:	Mobile country code. This info is publicly shared by International Telecommunication Union (link)

- **MNC**:	Mobile network code. This info is publicly shared by International Telecommunication Union (link)

- **AREA**:	Location Area Code

- **CELL**:	This is a unique number used to identify each Base transceiver station or sector of BTS

- **Longitude**:	Longitude, is a geographic coordinate that specifies the east-west position of a point on the Earth's surface

- **Latitude**:	Latitude is a geographic coordinate that specifies the north–south position of a point on the Earth's surface.

- **Range**:	Approximate area within which the cell could be. (In meters)

- **Samples**:	Number of measures processed to get a particular data point

- **Changeable=1**:	The location is determined by processing samples

- **Changeable=0**:	The location is directly obtained from the telecom firm

- **Created**:	When a particular cell was first added to database (UNIX timestamp)

- **Updated**:	When a particular cell was last seen (UNIX timestamp)

- **AverageSignal**:	To get the positions of cells, OpenCelliD processes measurements from data contributors. Each measurement includes GPS location of device + Scanned cell identifier (MCC-MNC-LAC-CID) + Other device properties (Signal strength). In this process, signal strength of the device is averaged. Most ‘averageSignal’ values are 0 because OpenCelliD simply didn’t receive signal strength values.


In [7]:
%%time
df = pd.read_csv('./opencellid_data/cell_towers_us.csv') # ./opencellid_data/cell_towers.csv if using the worldwide dataset

df.rename(columns={'net':'MNC', 'mcc':'MCC'}, inplace=True)

print(df.shape)

(7342373, 14)
CPU times: user 231 ms, sys: 169 ms, total: 400 ms
Wall time: 242 ms


In [8]:
df.head()

,radio,MCC,MNC,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
0,GSM,310,260,32192,22568,0,-73.858566,40.898438,1779,12,1,1459696254,1489125559,0
1,GSM,310,260,22629,61562,0,-95.876501,41.286475,3367,17,1,1459810860,1609205988,0
2,GSM,310,260,51052,44152,0,-71.084538,42.381555,1000,45,1,1459812328,1633182437,0
3,GSM,310,260,51051,40311,0,-71.085559,42.381356,1000,52,1,1459812328,1527695374,0
4,GSM,310,260,5973,60092,0,-95.570630,29.735371,1000,22,1,1459692332,1490851584,0


## Read mcc-mnc dataset to map to carrier name

Publicaly available dataset: https://mcc-mnc.net/ powered by [simbase](https://www.simbase.com/)


In [9]:
%%time
df_carriers = pd.read_csv('https://s3.amazonaws.com/mcc-mnc.net/mcc-mnc.csv', sep=';')[['MCC', 'MNC', 'Country', 'Operator']]

# rename Operator to Operator
df_carriers.rename(columns={'Operator': 'Operator'}, inplace=True)

CPU times: user 76.7 ms, sys: 9.03 ms, total: 85.8 ms
Wall time: 306 ms


,MCC,MNC,Country,Operator
0,901,27,Worldwide,Monaco Telecom
1,242,99,Norway,TampNet AS
2,363,2,Aruba,Digicel Aruba
3,412,40,Afghanistan,MTN Group Afghanistan
4,412,1,Afghanistan,Afghan Wireless Communication Company
...,...,...,...,...
2692,647,4,French Indian Ocean Territories (France),Zeop mobile
2693,204,64,Netherlands,Zetacom B.V.
2694,204,15,Netherlands,Ziggo B.V.
2695,214,10,Spain,"ZINNIA TELECOMUNICACIONES, S.L.U."


In [10]:
df_carriers.head()

,MCC,MNC,Country,Operator
0,901,27,Worldwide,Monaco Telecom
1,242,99,Norway,TampNet AS
2,363,2,Aruba,Digicel Aruba
3,412,40,Afghanistan,MTN Group Afghanistan
4,412,1,Afghanistan,Afghan Wireless Communication Company


In [11]:
%%time
df_final = df.merge(df_carriers, left_on=['MCC', 'MNC'], right_on=['MCC', 'MNC']).drop(columns=['MCC', 'MNC'])

CPU times: user 23.4 ms, sys: 14.1 ms, total: 37.5 ms
Wall time: 34.6 ms


In [12]:
%%time
df_final.head()

CPU times: user 3.4 ms, sys: 2.92 ms, total: 6.32 ms
Wall time: 4.97 ms


,radio,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,Country,Operator
0,GSM,32192,22568,0,-73.858566,40.898438,1779,12,1,1459696254,1489125559,0,United States of America,T-Mobile USA
1,GSM,22629,61562,0,-95.876501,41.286475,3367,17,1,1459810860,1609205988,0,United States of America,T-Mobile USA
2,GSM,51052,44152,0,-71.084538,42.381555,1000,45,1,1459812328,1633182437,0,United States of America,T-Mobile USA
3,GSM,51051,40311,0,-71.085559,42.381356,1000,52,1,1459812328,1527695374,0,United States of America,T-Mobile USA
4,GSM,5973,60092,0,-95.570630,29.735371,1000,22,1,1459692332,1490851584,0,United States of America,T-Mobile USA


### Analyzing Operator types

In [13]:
%%time
# Most reported cell towers by operators
df_final.Operator.value_counts()

CPU times: user 13.5 ms, sys: 13.1 ms, total: 26.6 ms
Wall time: 24.6 ms


Operator
T-Mobile USA                         2438095
AT&T Mobility                        2256562
Sprint Corporation                   1486370
Verizon Wireless                      981488
U.S. Cellular                          72391
                                      ...   
Iowa RSA No. 2 LP                          1
LigTel Communications                      1
Mark Twain Communications Company          1
New Dimension Wireless Ltd.                1
Tyntec Inc.                                1
Name: count, Length: 121, dtype: int64

In [14]:
%%time
# Categorizing Operators with less than 1000 cells to other to reduce clutter
operators_to_rename = df_final.Operator.value_counts()[df_final.Operator.value_counts()<=1000].index.tolist()
len(operators_to_rename)

CPU times: user 13.3 ms, sys: 8.05 ms, total: 21.3 ms
Wall time: 17.3 ms


86

In [15]:
%%time
# 11 s
df_final['Operator'] = df_final['Operator'].replace(operators_to_rename, 'Other')

CPU times: user 11.5 ms, sys: 8.06 ms, total: 19.6 ms
Wall time: 17.5 ms


In [16]:
%%time 
# 3 s
operators_df = df_final.groupby(['Country', 'Operator']).agg({'radio':'count'}).reset_index()

CPU times: user 8.04 ms, sys: 9.14 ms, total: 17.2 ms
Wall time: 14.6 ms


In [17]:
# Final operator analysis
operators_df

,Country,Operator,radio
0,American Samoa (USA),Other,1
1,Guam (USA),"IT&E Overseas, Inc",1863
2,Guam (USA),NTT DoCoMo Pacific,2379
3,Guam (USA),"Teleguam Holdings, LLC",722
4,Guam (USA),Wave Runner LLC,927
5,Northern Mariana Islands (USA),PTI Pacifica Inc.,1568
6,United States of America,700 MHz Public Safety Broadband,4848
7,United States of America,AT&T Mobility,2256562
8,United States of America,Americell PA 3 Partnership,2806
9,United States of America,Carolina West Wireless,1212


### Analyzing Radio types

In [24]:
# Analyzing radio cell tower types
pn.pane.HoloViews(
    df_final[["radio"]].value_counts().reset_index().hvplot.bar(title="radio types")
)

TypeError: a bytes-like object is required, not 'cupy.cuda.memory.MemoryPointer'

HoloViews(Bars, height=300, sizing_mode='fixed', width=700)

In [25]:
# Most LTE towers country wise
pn.pane.HoloViews(
    df_final[df_final.radio == 'LTE'].Country.value_counts().reset_index().head(10).hvplot.barh(x='Country')
)

TypeError: a bytes-like object is required, not 'cupy.cuda.memory.MemoryPointer'

HoloViews(Bars, height=300, sizing_mode='fixed', width=700)

In [26]:
%%time
columns = ['radio', 'area', 'lon', 'lat', 'range', 'samples', 'changeable', 'created', 'averageSignal', 'Country', 'Operator']
df_final = df_final[columns]

CPU times: user 811 μs, sys: 0 ns, total: 811 μs
Wall time: 823 μs


## Create an exploratory panel dashboard

In [27]:
%%time
# Convert 'radio' to a color value to display in the dashboard
color_map = {
    'UMTS': [255, 165, 0, 140],  # Orange
    'LTE': [255, 0, 0, 140],     # Red
    'GSM': [135, 206, 250, 140], # Light Blue
    'CDMA': [123, 104, 238, 140],# Medium Slate Blue
    'NR': [0, 128, 0, 140]       # Green
}
df_final['color'] = df_final['radio'].map(color_map)
df_final['created'] = pd.to_datetime(df['created'], unit='s').dt.year

column_view::get_data: Unsupported type: 24
CPU times: user 55 ms, sys: 12.4 ms, total: 67.4 ms
Wall time: 61.9 ms


In [28]:
# declare a legend
legend_markdown = """
### Radio Type Legend

- **UMTS**: <span style="height: 10px; width: 10px; background-color: rgba(255, 165, 0, 0.55); border-radius: 50%; display: inline-block;"></span>
- **LTE**: <span style="height: 10px; width: 10px; background-color: rgba(255, 0, 0, 0.55); border-radius: 50%; display: inline-block;"></span>
- **GSM**: <span style="height: 10px; width: 10px; background-color: rgba(135, 206, 250, 0.55); border-radius: 50%; display: inline-block;"></span>
- **CDMA**: <span style="height: 10px; width: 10px; background-color: rgba(123, 104, 238, 0.55); border-radius: 50%; display: inline-block;"></span>
- **NR**: <span style="height: 10px; width: 10px; background-color: rgba(0, 128, 0, 0.55); border-radius: 50%; display: inline-block;"></span>
"""

In [29]:
class CellTowerDashboard(param.Parameterized):
    # filter by radio cell tower type (GSM, LTE, UMTS, CDMA or all)
    radio_type = param.ListSelector(objects=df_final['radio'].unique().tolist(), default=[])
    # filter by radio operator type (TMobile, At&t, ...)
    operator = param.ListSelector(objects=df_final.Operator.value_counts().sort_values(ascending=False).index.tolist(), default=[])
    # country to display the cell-tower data points for
    country = param.ListSelector(objects=df_final.Country.value_counts().sort_values(ascending=False).index.tolist(), default=[])
    # Filter by date the tower was first reported
    date_range = param.Range(bounds=(df_final['created'].min(), df_final['created'].max()), step=1, default=(df_final['created'].min(), df_final['created'].max()))

    # Since rendering is equal for both pandas and cudf.pandas, we track the compute progress where cudf.pandas is much more efficient
    compute_progress = param.Integer(default=0, bounds=(0, 100))

    # point_budget is used to curtail number of points rendered at any time. Can be adjusted as per client browser capabilities
    point_budget = param.Integer(default=50_000, bounds=(10000,100_000), step=10_000)

    selected_data_size = param.Integer(default=df_final.shape[0])
    df = param.DataFrame(default=df_final.sample(50_000))
    execution_time = param.Number(default=0, doc="Execution Time")

    @param.depends('radio_type', 'operator', 'date_range', 'country', 'point_budget', watch=True)
    def filter_dataframe(self):
        start_time = time.time()  # Start time
        self.compute_progress = 0
        filtered_indices = df_final['created'].between(self.date_range[0], self.date_range[1])
        self.compute_progress = 25
        if self.radio_type:
            filtered_indices &= df_final['radio'].isin(self.radio_type)
        self.compute_progress = 50
        if self.operator:
            filtered_indices &= df_final['Operator'].isin(self.operator)
        self.compute_progress = 75
        if self.country:
            filtered_indices &= df_final['Country'].isin(self.country)
        self.compute_progress = 100

        filtered_df = df_final[filtered_indices]

        self.selected_data_size = filtered_df.shape[0]
        if filtered_df.shape[0] > self.point_budget:
            filtered_df = filtered_df.sample(n=self.point_budget)

        end_time = time.time()  # End time
        self.execution_time = end_time - start_time  # Compute execution time

        self.df = filtered_df


    @param.depends('df')
    def update_scatter_plot(self):
        COUNTRIES = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_scale_rank.geojson"
        # Define the layer
        layers = [
            pdk.Layer(
                "GeoJsonLayer",
                id="base-map",
                data=COUNTRIES,
                stroked=False,
                filled=True,
                get_fill_color=[200, 200, 200],
            ),
            pdk.Layer(
            'ScatterplotLayer',  # Use ScatterplotLayer
            self.df,
            pickable=True,
            opacity=0.8,
            stroked=True,
            filled=True,
            radius_scale=6,
            radius_min_pixels=1,
            radius_max_pixels=10,
            line_width_min_pixels=1,
            get_position=['lon', 'lat'],
            radius=1000,
            get_line_color="color"
        )]
        # Set the viewport location
        view_state = pdk.ViewState(latitude=self.df.lat.median(), longitude=self.df.lon.median(), zoom=0, bearing=0, pitch=0)
        view = pdk.View(type="_GlobeView", controller=True)
        # Render
        return pn.Column("### Visualized Data Points", self.visualizedDataSize,
                pn.pane.DeckGL(pdk.Deck(views=[view], layers=layers, initial_view_state=view_state, map_provider=None)))

    @param.depends('selected_data_size')
    def computedDataSize(self):
        return pn.widgets.Number(value=self.selected_data_size, format="{value:,}", font_size='25pt')


    @param.depends('execution_time')
    def executionTime(self):
        return pn.pane.Markdown(f"### Compute Time: {self.execution_time:.2f} seconds")

    @param.depends('point_budget', 'selected_data_size')
    def visualizedDataSize(self):
        visualized_points = min(self.selected_data_size, self.point_budget)
        return pn.widgets.Number(value=visualized_points, format="{value:,}", font_size='15pt')

    @param.depends('df')
    def update_bar_charts(self):
        radio_count = self.df['radio'].value_counts().head().hvplot.barh(title='Radio Type Count', responsive=True, tools=['hover']).opts(default_tools=[])
        operator_count = self.df['Operator'].value_counts().head().hvplot.barh(title='Operator Tower Count', responsive=True, tools=['hover']).opts(default_tools=[])
        return pn.Column(radio_count, operator_count)

    @param.depends('compute_progress', 'execution_time')
    def compute_bar(self):
        return pn.widgets.Progress(value=self.compute_progress,active=False, sizing_mode='stretch_width')

    @param.depends('df')
    def view(self):
        box = pn.GridSpec(ncols=5, width=1400, height=600)
        widgets = pn.Column(
            pn.WidgetBox(
                "### Queried Data Points",
                self.computedDataSize,
                pn.Column(
                    pn.Row(
                        self.executionTime,
                        pn.widgets.TooltipIcon(value="While the rendering process remains consistent between pandas and cudf, our focus is on the significant reduction in compute time achieved with cudf.pandas"),
                    ),
                    self.compute_bar),
                sizing_mode="stretch_width"
            ),
            pn.WidgetBox("### Query Filters",
                pn.Param(self.param.radio_type, widgets={'radio_type': {"type": pn.widgets.MultiChoice, "sizing_mode": "stretch_width"}}),
                pn.Param(self.param.operator, widgets={'operator': {"type": pn.widgets.MultiChoice, "sizing_mode": "stretch_width"}}),
                pn.Param(self.param.country, widgets={'country': {"type": pn.widgets.MultiChoice, "sizing_mode": "stretch_width"}}),
                pn.Param(self.param.date_range, widgets={'date_range': {"type": pn.widgets.RangeSlider, "throttled": True, "sizing_mode": "stretch_width"}}),
                pn.Param(self.param.point_budget, widgets={'point_budget': {"type": pn.widgets.IntSlider, "throttled": True, "sizing_mode": "stretch_width"}}),
                sizing_mode="stretch_width"
            )
        )

        box[:,0] = widgets
        box[:,1:3] = self.update_scatter_plot
        box[:,3:5] = self.update_bar_charts
        return box

column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24


In [30]:
dashboard = CellTowerDashboard(point_budget=10_000)
dashboard.view()

TypeError: a bytes-like object is required, not 'cupy.cuda.memory.MemoryPointer'

GridSpec(height=600, ncols=5, nrows=1, sizing_mode='fixed', width=1400)
    [0] Column(height=600, width=280)
        [0] WidgetBox(sizing_mode='stretch_width')
            [0] Markdown(str)
            [1] ParamMethod(method, _pane=Number, defer_load=False)
            [2] Column
                [0] Row
                    [0] ParamMethod(method, _pane=Markdown, defer_load=False)
                    [1] TooltipIcon(value='While the rendering proce...)
                [1] ParamMethod(method, _pane=Progress, defer_load=False)
        [1] WidgetBox(sizing_mode='stretch_width')
            [0] Markdown(str)
            [1] Param(CellTowerDashboard, parameters=['radio_type'], show_name=False, widgets={'radio_type': {'type': <c...})
            [2] Param(CellTowerDashboard, parameters=['operator'], show_name=False, widgets={'operator': {'type': <cla...})
            [3] Param(CellTowerDashboard, parameters=['country'], show_name=False, widgets={'country': {'type': <clas...})
            [4]

Dashboard Screenshot:

![image.png](attachment:6a1c3200-eb9c-4a01-85d6-a18be7afa66e.png)